# revelance ranking

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer, util
tqdm.pandas()

In [2]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
df = pd.read_json('data_gen/wikipedia_20220620_cleaned_parsed.jsonl',lines=True)

In [4]:
corpus_embeddings = pickle.load(open('data_gen/corpus_embeddings.pkl','rb'))
title_embeddings = pickle.load(open('data_gen/title_embeddings.pkl','rb'))

In [5]:
title_embeddings.shape , corpus_embeddings.shape

(torch.Size([1273469, 768]), torch.Size([1273469, 384]))

In [6]:
title_model = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')
sentence_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

No sentence-transformers model found with name C:\Users\ngoph/.cache\torch\sentence_transformers\VoVanPhuc_sup-SimCSE-VietNamese-phobert-base. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\ngoph/.cache\torch\sentence_transformers\VoVanPhuc_sup-SimCSE-VietNamese-phobert-base were not used when initializing RobertaModel: ['mlp.dense.weight', 'mlp.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
from transformers import pipeline

In [8]:
tc = pipeline('text-classification', model='model', tokenizer='bert-base-uncased', device=0)

In [9]:
qa = pipeline('question-answering', model='model_2\checkpoint-44454',device=0)

In [10]:
tc('Chủ tịch nước Việt Nam là ai?')

[{'label': 'LABEL_0', 'score': 0.9997279047966003}]

In [11]:
# model_1 = tf.keras.models.load_model('model_1')
LABEL={
	'LABEL_0':'wiki',
	'LABEL_1':'number',
	'LABEL_2':'text'
}

# inference

In [12]:
import json
import re

In [13]:
with open('data/zac2022_testa_only_question.json',encoding='utf-8') as f:
    data=json.load(f)
    reference_df = pd.json_normalize(data,'data')

In [14]:
reference_df['question'].tolist()

['Đạo diễn phim Titanic là ai',
 'Tổng thống Hoa Kỳ thứ 45 là ai',
 'Hiện nay ai là tổng bí thư nước Việt Nam',
 'đâu là bản hiến pháp lâu đời nhất thế giới',
 'Tổ chức thống nhất châu Phi được thành lập ở đâu',
 'Ngọn núi lửa cao nhất thế giới Ojos del Salado nằm ở đâu',
 'Tác giả của "Bạch Đằng giang phú"',
 'Ai là người đặt tên nước là Vạn Xuân?',
 'Vị vua nào đã đặt tên quốc hiệu nước ta là Đại Cồ Việt',
 'Trương Quang Nghĩa sinh ngày bao nhiêu',
 'Hồ Chí Minh đọc bản Tuyên ngôn Độc lập vào ngày tháng năm nào',
 'Chủ tịch Hội sinh viên Việt Nam tại Paris nhiệm kỳ 1965-1966 là ai?',
 'Tây Du Ký phiên bản Trương Kỷ Trung khởi quay năm nào',
 'Ngày môi trường thế giới 5 tháng 6 bắt đầu từ năm nào',
 'Sau khi qua đời, vua Lý Nhân Tông truyền ngôi cho ai',
 'Tỉnh nào có trữ lượng dầu mỏ lớn nhất Việt Nam',
 'Adolf Hitler sinh vào năm nào',
 'Thủ đô của Đông Ti Mo ?',
 'An Tư công chúa là con của vua Trần nào',
 'Thuyết tương đối đặc biệt và thuyết tương đối tổng quát là ai phát minh ra?

In [15]:
def date_format(answer):
    if re.search(r'ngày \d{1,2} tháng \d{1,2} năm \d{4}',answer):
        answer = re.search(r'ngày \d{1,2} tháng \d{1,2} năm \d{4}',answer).group()
    # ngay xx thang xx
    elif re.search(r'ngày \d{1,2} tháng \d{1,2}',answer):
        answer = re.search(r'ngày \d{1,2} tháng \d{1,2}',answer).group()
    # thang xx nam xxx
    elif re.search(r'tháng \d{1,2} năm \d{4}',answer):
        answer = re.search(r'tháng \d{1,2} năm \d{4}',answer).group()
    # nam xxxx
    elif re.search(r'năm \d{4}',answer):
        answer = re.search(r'năm \d{4}',answer).group()

    # 1/1/2021  dd{token}mm{token}yyyy
    elif re.search(r'\d{1,2}[ /|-]\d{1,2}[ /|-]\d{4}',answer):
        answer = re.search(r'\d{1,2}[ /|-]\d{1,2}[ /|-]\d{4}',answer).group()
        token = re.search(r'[ /|-]',answer).group()
        answer = answer.split(token)
        answer = 'ngày '+answer[0]+' tháng '+answer[1]+' năm '+answer[2]
    # 1/1       dd{token}mm
    elif re.search(r'\d{1,2}[ /|-]\d{1,2}',answer):
        answer = re.search(r'\d{1,2}[ /|-]\d{1,2}',answer).group()
        token = re.search(r'[ /|-]',answer).group()
        answer = answer.split(token)
        answer = 'ngày '+answer[0]+' tháng '+answer[1]
    # 1/2021    mm{token}yyyy
    elif re.search(r'\d{1,2}[ /|-]\d{4}',answer):
        answer = re.search(r'\d{1,2}[ /|-]\d{4}',answer).group()
        token = re.search(r'[ /|-]',answer).group()
        answer = answer.split(token)
        answer = 'tháng '+answer[0]+' năm '+answer[1]
    return answer

In [16]:
answer = 'asd á ngày 13 tháng 9 năm 1943 abc d'
answer = '3 10 2021'
date_format(answer)

'ngày 3 tháng 10 năm 2021'

In [17]:
def wiki_format(answer):
    title_embedding = title_model.encode( answer  ,convert_to_tensor=True)
    hits = util.semantic_search(title_embedding, title_embeddings, top_k=1)[0]
    best = hits[0]
    best_title = df.iloc[best['corpus_id']]['title']
    answer = 'wiki/'+best_title.replace(' ','_')
    return answer

In [18]:
wiki_format('James Cameron')

'wiki/James_Cameron'

In [19]:
def format_answer(answer,type):
    if type=='wiki':
        answer = wiki_format(answer)
    if type == 'number':
        answer = re.sub(r'\D', '', answer)
    if type == 'text':
        answer = answer.lower()
        answer = date_format(answer)
    return answer

In [20]:
random_question = reference_df['question'].sample(1).values[0]
query_embedding = sentence_model.encode(random_question,convert_to_tensor=True)
print(random_question)
q_type = tc(random_question)[0]['label']
print(LABEL[q_type])
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=10)[0]

answers =[]
for hit in hits:
    corpus_id = hit['corpus_id']
    doc_score = hit['score']

    title = df.iloc[corpus_id]['title']
    text =  df.iloc[corpus_id]['text']
    text = re.sub(r'\s+',' ',text)
    text= re.sub(r'_',' ',text)
    answer = qa(question=random_question,context=text)
    answer_score = answer['score']
    answers.append((title,answer['answer'],doc_score,answer_score))

sorted_answers = sorted(answers,key=lambda x:x[2]+x[3],reverse=True)

for title,answer,doc_score,answer_score in sorted_answers:
    print('Title: ',title)
    print('Answer: ',answer)
    print('Format answer:',format_answer(answer,q_type))
    print('Doc score: ',doc_score)
    print('Answer score: ',answer_score)
    print('Calc score:',doc_score+answer_score)
    print('-----------------------')

Bộ phim " Rừng Phương Nam " được chuyển thể từ truyện ngắn nào
wiki
Title:  Đất rừng phương Nam
Answer:  miền Tây Nam Bộ
Format answer: miền Tây Nam Bộ
Doc score:  0.5703535676002502
Answer score:  0.998971164226532
Calc score: 1.5693247318267822
-----------------------
Title:  Tell Me A Lie
Answer:  Tell Me A Lie
Format answer: Tell Me A Lie
Doc score:  0.5741838216781616
Answer score:  0.5046601891517639
Calc score: 1.0788440108299255
-----------------------
Title:  Chuyện rừng xanh
Answer:  Chuyện rừng xanh
Format answer: Chuyện rừng xanh
Doc score:  0.6099864840507507
Answer score:  4.623032512363068e-10
Calc score: 0.609986484513054
-----------------------
Title:  Mưa rừng
Answer:  Thúy Nga
Format answer: Thúy Nga
Doc score:  0.574583888053894
Answer score:  1.237227160999499e-11
Calc score: 0.5745838880662664
-----------------------
Title:  Tiền truyện
Answer:  Tiền truyện
Format answer: Tiền truyện
Doc score:  0.567855954170227
Answer score:  2.0758415331719599e-13
Calc score: 0

In [21]:
top_k = 5
submission = []
for row in tqdm(reference_df.iterrows(),total=len(reference_df)):
    question = row[1]['question']
    query_embedding = sentence_model.encode(question,convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=10)[0]

    answers =[]
    for hit in hits:
        corpus_id = hit['corpus_id']
        doc_score = hit['score']
        title = df.iloc[corpus_id]['title']
        text =  df.iloc[corpus_id]['text']
        text = re.sub(r'\s+',' ',text)
        text= re.sub(r'_',' ',text)
        answer = qa(question=question,context=text)
        answer_score = answer['score']
        answers.append((title,answer['answer'],doc_score,answer_score))
    sorted_answers = sorted(answers,key=lambda x:x[2]+x[3],reverse=True)

    best_result = sorted_answers[0]
    answer = best_result[1]
    type = LABEL[tc(question)[0]['label']]
    answer = format_answer(answer,type)
    submission.append({
        'id': row[1]['id'],
        'question': question,
        'answer': answer,
        'raw_answer': best_result[1],
        'type': type
    })

  0%|          | 0/600 [00:00<?, ?it/s]d:\Software\Anaconda\envs\rs\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 10%|█         | 62/600 [04:09<36:08,  4.03s/it]  


KeyboardInterrupt: 

In [22]:
df_submission = pd.DataFrame(submission)

In [23]:
df_submission

,id,question,answer,raw_answer,type
0,testa_1,Đạo diễn phim Titanic là ai,wiki/Ana_Corradi,Orlando Corradi,wiki
1,testa_2,Tổng thống Hoa Kỳ thứ 45 là ai,wiki/Benjamin_Franklin,Benjamin Franklin,wiki
2,testa_3,Hiện nay ai là tổng bí thư nước Việt Nam,wiki/Lê_Đức_Thọ,Lê Đức Thọ,wiki
3,testa_4,đâu là bản hiến pháp lâu đời nhất thế giới,wiki/Ba_Lan,Ba Lan,wiki
4,testa_5,Tổ chức thống nhất châu Phi được thành lập ở đâu,wiki/Addis_Ababa,"Addis Ababa, Ethiopia",wiki
...,...,...,...,...,...
57,testa_58,kỳ trăng tròn thứ hai trong một tháng được gọi...,wiki/Trăng_xanh,trăng xanh,wiki
58,testa_59,Thủ đô của Hàn Quốc hiện nay là gì,wiki/Seoul,Seoul,wiki
59,testa_60,"Tam tòng, tứ đức là chuẩn mực của người phụ nữ...",wiki/Thế_giới_phương_Đông,phương Đông,wiki
60,testa_61,Sự kiện Thiên An Môn diễn ra vào năm nào,năm bính tý,năm Bính Tý,text


In [ ]:
from datetime import datetime
date = datetime.now().strftime("%d_%m_%Y_%H_%M")

In [ ]:
import os 
os.makedirs('submissions',exist_ok=True)
os.makedirs('submissions/'+date,exist_ok=True)

In [ ]:
json_data = df_submission.to_dict(orient='records')
with open(f'submissions/{date}/submission_raw.json', 'w', encoding='utf-8') as f:
    json.dump({'data': json_data}, f, ensure_ascii=False, indent=2)

In [ ]:
df_submission.drop(['raw_answer','type'],axis=1,inplace=True)

In [ ]:
json_data = df_submission.to_dict(orient='records')
with open(f'submissions/{date}/submission.json', 'w', encoding='utf-8') as f:
    json.dump({'data': json_data}, f, ensure_ascii=False, indent=2)